## 연속형

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import scipy 
scipy.__version__
import scipy.stats as stats

In [2]:
df_TOCC = pd.read_csv('../../datasets/TypeOfContractChannel.csv')
df_TOCC[:3]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0


### x:연속형, y:연속형
    - 상관관계 확인 (corr?)
            - coefficient (상관관계)_계측치? : 상관관계 <= -1   상관관계 <= 1
                ⇒ 0에 가까우면 관계 없음. -1과 가까워질수록 반비례/ 1과 가까워질수록 비례 

#### 선정변수 
        1. 'age' (연속), 'Mileage' (연속)
            → 분석 내용: 나이에 따른 마일리지 변화?
                - 귀무가설: 나이에 따른 마일리지 변화 없음.
                - 대립가설: 나이에 따른 마일리지 변화 있음.
            → 분석 결론 (p-value : 0.05 ?), stats.pearsonr('비교1', '비교2')
                ⇒ PearsonRResult(statistic=0.9732809346109912, pvalue=0.0)
                - 통계 결론: statistic_0.97, p-value < 0.05 이므로 대립가설 참
                - 사실 결론: 나이에 따른 마일리지 변화 있다.
            → correlation의 유의미함을 보려면 pearsonr을 사용한다.   

In [3]:
df_TOCC[['age','Mileage']].describe()
#count, mean, 50% 확인하면됨

,age,Mileage
count,40509.000000,40509.000000
mean,46.608285,1967.922412
std,13.465977,436.707329
min,20.000000,872.000000
25%,37.000000,1646.000000
50%,46.000000,1946.000000
75%,56.000000,2263.000000
max,107.000000,3803.000000


In [4]:
df_TOCC[['age']].count()

age    40509
dtype: int64

In [5]:
df_TOCC[['Mileage']].count()

Mileage    40509
dtype: int64

In [6]:
df_age_Mileage = df_TOCC[['age','Mileage']]
df_age_Mileage[:2]

,age,Mileage
0,43.0,1862.0
1,62.0,2532.0


In [7]:
df_TOCC[['age','Mileage']].corr()

,age,Mileage
age,1.000000,0.973281
Mileage,0.973281,1.000000


In [8]:
df_age_Mileage.isnull().sum()

age        10795
Mileage    10795
dtype: int64

In [9]:
df_age_Mileage = df_age_Mileage.dropna().copy()

#### corr() 확인했을때, age-Mileage:0.973281 
    → 분포적으로 유의미한건가?, 어떤 가설이 맞는가?
    ⇒ correlation의 p-value를 보고자 확인: pearsonr 확인. 

In [10]:
df_age_Mileage.isnull().sum()

age        0
Mileage    0
dtype: int64

In [11]:
df_age_Mileage.corr()

,age,Mileage
age,1.000000,0.973281
Mileage,0.973281,1.000000


In [12]:
stats.pearsonr(df_age_Mileage['age'], df_age_Mileage['Mileage'])

PearsonRResult(statistic=0.9732809346109912, pvalue=0.0)

    -> pearson에서 statistic = 0.97.. 이거의 음과 양의 수로 비례인지 반비례인지 확인 가능
    -> p-value= 0.0 으로 보아 유의미함

In [13]:
df_TOCC[['type_of_contract','cancellation']].describe()
# 항목 개수 확인

,type_of_contract,cancellation
count,51300,51279
unique,2,2
top,렌탈,정상
freq,46481,50372


In [14]:
df_TOCC_category = df_TOCC[['type_of_contract','cancellation']]
df_TOCC_category[:2]

,type_of_contract,cancellation
0,렌탈,정상
1,렌탈,정상


In [15]:
# index = groupby= y = 목표변수
df_TOCC_category_pivot = pd.pivot_table(data=df_TOCC_category, index=['type_of_contract'], columns=['cancellation'], aggfunc=(len))
df_TOCC_category_pivot

cancellation,정상,해약
type_of_contract,,
렌탈,46137,344
멤버십,4231,563


In [16]:
import scipy 
scipy.__version__
import scipy.stats as stats

- chi2_contingency(observed[, correction, lambda_])
- observedarray_like
The contingency table. The table contains the observed frequencies (i.e. number of occurrences) in each category. In the two-dimensional case, the table is often described as an “R x C table”.

In [17]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True)

Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))

In [18]:
df_TOCC[['state','cancellation']].describe()

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372
